In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

import gzip
import pickle

np.random.seed(25)

#importing mnist data from local .pkl file
#encoding latin1 as due to version challenges. faced UnicodeDecodeError

with gzip.open('mnist.pkl.gz','rb') as f:
    (X_train, y_train), (X_test, y_test) = pickle.load(f,encoding='latin1')

print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)    

X_train = X_train.reshape(X_train.shape[0],28,28,1)
X_test = X_test.reshape(X_test.shape[0],28,28,1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train/=255
X_test/=255

X_train.shape
X_test.shape

number_of_classes = 10

Y_train = np_utils.to_categorical(y_train,number_of_classes)
Y_test = np_utils.to_categorical(y_test,number_of_classes)

y_train.shape
y_train[0]
Y_train.shape
Y_train[0]


#till now, we have created the input X and output Y in desired format. Now lets build the CNN network


Each convolution layer correspond to three steps:
 1. Convolution
 2. Activation
 3. Pooling

 Repeating 1,2,3 again will add more convolution layers..

Input of CNN is WxHxD
here it 28x28x1

output will be based on 
num of filters, dimension of filter, padding, etc..





In [ ]:
model = Sequential()

model.add(Conv2D(32,(3,3),input_shape = (28,28,1)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

BatchNormalization(axis=-1)
model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())

BatchNormalization()
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormaliation()
model.add(Dropout(0.2))
model.add(Dense(10))

model.add(Activation('softmax'))

model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])



In [ ]:
# generting more images by rotating the existing ones..

gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)

test_gen = ImageDataGenerator()


train_generator = gen.flow(X_train, Y_train, batch_size=64)
test_generator = test_gen.flow(X_test, Y_test, batch_size=64)

# model.fit(X_train, Y_train, batch_size=128, nb_epoch=1, validation_data=(X_test, Y_test))

model.fit_generator(train_generator, steps_per_epoch=60000//64, epochs=5, 
                    validation_data=test_generator, validation_steps=10000//64)


predictions = model.predict_classes(X_test)

predictions = list(predictions)
actuals = list(y_test)

sub = pd.DataFrame({'Actual': actuals, 'Predictions': predictions})
sub.to_csv('./output_cnn.csv', index=False)

score = model.evaluate(X_test, Y_test)